# Collaboration and Competition Project Report

# 1. Learning Algorithm

### Agent
In this project, Deep Deterministic Policy Gradient (DDPG) algorithm is used for training the agent. The agent contains information about the state size and number of actions to choose from. The agent also contains one local actor network, one target actor network, one local critic network and one target critic network. The networks are imported from the `model.py` file.

The environment has 2 players, each player observe state from their own perspective. Therefore, we used a single agent for both players. We also used one shared replay buffer instead of keeping one for each player. 

### Actor and critic network

The networks are based on Pytorch. 

For the actor network, we used a neural network with 2 hidden layers. The input to the network is the state variable which has dimension of `state_size`. The 2 hidden layers have 400 and 300 units respectively. After each hidden layer, relu is used as the activation function. The output layer has dimension of `action_size` and represent the action that will be taken. 

For the critic network, we used a neural network with 2 hidden layers. The input to the network is the state variable which has dimension of `state_size`. The first hidden layer has 400 units and use relu as the activation function. The output of this layer is concatenated with a action value before being put into the second hidden layer with 300 units and relu as the activation function. The output layer has dimension of 1 and represent the state-action value.

### Environment
The environment is based on UnityEnvironment, for more description please see the `README.md` file.

### Learning algorithm
The training is perfomed in an episodic way. At the start of each episode, the environment is reset to initial state.

1. The current state for each player is passed to the local actor network and an action is returned for each player. A noise is added to the returned action to make the action a little more exploratory.  

2. The returned actions are passed to the environment, the environment determines the next state, the reward of this step and a flag indicating whether this episode is finished or not for each player. The (state, action, reward, next state, done) tuple is called an experience and the 2 experiences from both players are stored in the shared replay memory.

3. After every certain number of steps, the networks will update themselves. The agent samples a batch of experiences from the replay memory. The critic network is updated based on temporal difference method. We are trying to minimize the difference between the estimated current state-action value from local critic network and the sum of current reward and estimated next state-action value from target critic network. At this step, only local critic network is updated. The actor network is updated by trying to maximize the estimated state-action value from local critic network. At this step, only local actor network is updated. Note that the agent represents both players. 

4. The target actor and critic network are soft-updated: θ_target = τ*θ_local + (1 - τ)*θ_target

5. The score and the current state of the current episode is updated.

6. Repeat 1-5 until the episode finishes.

During training, we keep track of the most recent 100 episodes and we stop training when the average score of the 100 episodes reaches 0.5

### Noise function
A modified version of the Ornstein-Uhlenbeck noise was used when adding noise to the action value returned by the actor network. The original uniform distribution for the sigma term is replaced by a normal distribution N(0,1). For some reason this change has made training more stable. 


### Hyperparameters

The buffer size for the memory replay is 1e5, batch size for a batch of experiences is 128, discount factor for the next step is 0.99, tau for soft update of target parameters is 1e-3, learning rate of the Adam optimizer is 1e-4 for the actor network and 1e-3 for the critic network, the network is updated every 5 steps. 






# 2. Plot of Rewards

The environment is solved in 2303 episodes with an average score of 0.51 between episode 2303 to 2403. A plot of rewards is shown below 

<img src="https://raw.githubusercontent.com/bochaozhao/udacity_DRL/master/Collaboration_and_Competition/plot_of_scores.png" width="700" height="700"/>

# 3. Ideas for Future Work

1. Update frequency can be optimized. The training is very sensitive to update frequency and will almost always crash if the nwtworks are updated too frequently. For optimum performance and speed, update frequency and learning rate should be tuned together. Learning rate decay can also be experimented to see if it will help stabilize training. 

2. The noise function can be optimized. Modfiying the OU noise as shown above made a significant difference in agent performance. The reason is unclear. 

3. Prioritized Experience Replay can be used to improve the agent's performance. Each experience will be assigned a priority value based on the error in the critic network. The experiences with larger error will be sampled at a higher priority (probability). Sum Tree data structure can be used for prioritized experience replay.